In [1]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
!pip install selenium-stealth

# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument("--window-size=1920,1080")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("--proxy-server='direct://'")
options.add_argument("--proxy-bypass-list=*")
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--disable-blink-features")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument('ignore-certificate-errors')
options.add_experimental_option('useAutomationExtension', False)
options.add_experimental_option("excludeSwitches", ["enable-automation"])

'apt-get' is not recognized as an internal or external command,
operable program or batch file.
'apt' is not recognized as an internal or external command,
operable program or batch file.
'cp' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import re
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium_stealth import stealth
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm.auto import tqdm
import numpy as np
import requests
import json

In [3]:
ser = Service("C:\\chromedriver\chromedriver.exe")

In [4]:
driver = webdriver.Chrome(service=ser, options=options)

stealth(
    driver,
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36',
    languages = ["en-US", "en"],
    vendor = "Google Inc.",
    platform = "Win32",
    webgl_vendor = "Intel Inc.",
    renderer = "Intel Iris OpenGL Engine",
    fix_hairline = False,
    run_on_insecure_origins = False,
)

In [5]:
#Import Dataset
df_src = pd.read_csv('https://docs.google.com/spreadsheets/d/' +
                   '1U0n5tunbsXHvBRi_r5d3kuSXoJy_tutDxhOe2TiOwqM' +
                   '/export?gid=1390430554&format=csv')

In [6]:
df_src['url'][28:29]

28    https://lpse.balangankab.go.id/eproc4
Name: url, dtype: object

In [7]:
def month(var):
  var = var.split(' ')
  if "Januari" in var[1]:
    var[1] = "01"
    return var[0]+'-'+var[1]+'-'+var[2]
  elif "Februari" in var[1]:
    var[1] = "02"
    return var[0]+'-'+var[1]+'-'+var[2]
  elif "Maret" in var[1]:
    var[1] = "03"
    return var[0]+'-'+var[1]+'-'+var[2]
  elif "April" in var[1]:
    var[1] = "04"
    return var[0]+'-'+var[1]+'-'+var[2]
  elif "Mei" in var[1]:
    var[1] = "05"
    return var[0]+'-'+var[1]+'-'+var[2]
  elif "Juni" in var[1]:
    var[1] = "06"
    return var[0]+'-'+var[1]+'-'+var[2]
  elif "Juli" in var[1]:
    var[1] = "07"
    return var[0]+'-'+var[1]+'-'+var[2]
  elif "Agustus" in var[1]:
    var[1] = "08"
    return var[0]+'-'+var[1]+'-'+var[2]
  elif "September" in var[1]:
    var[1] = "09"
    return var[0]+'-'+var[1]+'-'+var[2]
  elif "Oktober" in var[1]:
    var[1] = "10"
    return var[0]+'-'+var[1]+'-'+var[2]
  elif "November" in var[1]:
    var[1] = "11"
    return var[0]+'-'+var[1]+'-'+var[2]
  elif "Desember" in var[1]:
    var[1] = "12"
    return var[0]+'-'+var[1]+'-'+var[2]
  else:
    return ''

def not_found():
    alasan_list.append('')
    kode_rup_list.append('')
    tanggal_pembuatan_list.append('')
    satuan_kerja_list.append('')
    sumber_dana_list.append('')
    nilai_pagu_list.append('')
    nilai_hps_list.append('')
    jenis_kontrak_list.append('')
    lokasi_pekerjaan_list.append('')
    kualifikasi_usaha_list.append('')
    syarat_kualifikasi_list.append('')
    jumlah_peserta_tender_list.append('')

def nilai_pengumuman(data_pengumuman, tahun_null):
  tr = data_pengumuman.find('table')
  
  #variabel alasan pembatalan dan alasan ulang
  try:
    pembatalan = tr.find(text='Alasan Pembatalan').find_next('td').text.strip()
  except:
    pembatalan = ''
  try:
    ulang = tr.find(text='Alasan di ulang').find_next('td').text.strip()
  except:
    ulang = ''
  if pembatalan or ulang:
    alasan = pembatalan + ', ' + ulang
    alasan_list.append(alasan)
  else:
    alasan_list.append('')

  #variabel kode rup
  try:
    rup = tr.find(text='Kode RUP').find_next('td').text.strip()
    kode_rup_list.append(rup)
  except:
    kode_rup_list.append('')

  tanggal_pembuatan = tr.find(text='Tanggal Pembuatan').find_next('td').text
  date = month(tanggal_pembuatan)
  tanggal_pembuatan_list.append(date)

  satker = tr.find(text='Satuan Kerja').find_next('td').text.strip()
  satuan_kerja_list.append(satker)

  metode = tr.find(text=['Metode Pengadaan','Sistem Pengadaan']).find_next('td').text.strip()

  metode_pengadaan = metode.split(' - ')[1]
  metode_pengadaan_list.append(metode_pengadaan)

  metode_evaluasi = metode.split(' - ')[2]
  metode_evaluasi_list.append(metode_evaluasi)

  ta = tr.find(text='Tahun Anggaran').find_next('td').text.strip()
  sumber_dana = ta.split(' ')[0]
  sumber_dana_list.append(sumber_dana)

  #variabel nilai pagu
  try:
    nilai_pagu = tr.find(text='Nilai Pagu Paket').find_next('td').text.strip().replace('Rp. ', '').replace('Rp ','')
    nilai_pagu_list.append(nilai_pagu)
  except:
    nilai_pagu_list.append('')

  #variabel nilai hps
  try:
    nilai_hps = tr.find(text='Nilai HPS Paket').find_next('td').text.strip().replace('Rp. ', '').replace('Rp ','')
    nilai_hps_list.append(nilai_hps)
  except:
    nilai_hps_list.append('')

  #variabel jenis kontrak
  try:
    cara_bayar = tr.find(text='Jenis Kontrak').find_next('td').text.strip()
    jenis_kontrak_list.append(cara_bayar)
  except:
    jenis_kontrak_list.append('')

  lokasi_pekerjaan = tr.find(text='Lokasi Pekerjaan').find_next('td').text.strip()
  lokasi_pekerjaan_list.append(lokasi_pekerjaan)

  #variabel kualifikasi usaha
  try:
    kualifikasi_usaha = tr.find(text='Kualifikasi Usaha').find_next('td').text.strip()
    kualifikasi_usaha_list.append(kualifikasi_usaha)
  except:
    kualifikasi_usaha_list.append('')

  syarat_kualifikasi = tr.find(text='Syarat Kualifikasi').find_next('td')
  syarat_kualifikasi_list.append(syarat_kualifikasi)

  peserta_tender = tr.find(text='Peserta Tender').find_next('td').text.strip()
  jumlah_peserta_tender_list.append(peserta_tender)
  
  if tahun_null:
    thn = tanggal_pembuatan.split()[-1]
    tahun_anggaran_list.append(thn)

In [24]:
def get_data_halaman_tender():
    data_tender = BeautifulSoup(driver.page_source, 'lxml')
    # looping to get main data from one url
    for area in tqdm(data_tender.find_all('tr', {'class': ['odd','even']})):
          kode_paket = area.find('td', class_="sorting_1").text
            
          if kode_paket in kode_paket_list:
              continue
          else:
              kode_paket_list.append(kode_paket)
              
              lpse = link_lpse.replace('/lelang', '')
              link_lpse_list.append(lpse)

              nama_paket = area.find('a').contents[0]
              nama_paket_list.append(nama_paket)

              status = [e.text for e in area.find_all('span')]
              status_list.append(status)

              detail_paket = area.select_one('p:nth-of-type(2)').text.strip()
              jenis_pengadaan = detail_paket.split(' - ')[0]
              jenis_pengadaan_list.append(jenis_pengadaan)

              tahun_anggaran= detail_paket.split(' - ')[1].replace('TA ','')
              tahun = tahun_anggaran.split(',')[0]
              tahun_null = False
              if tahun == 'null':
                tahun_null = True
              else:
                tahun_anggaran_list.append(tahun)

              metode_pemilihan = detail_paket.split(' - ')[2]
              metode_pemilihan_list.append(metode_pemilihan)

              nilai_kontrak = area.select_one('p:nth-of-type(3)').text.replace("Nilai Kontrak : ", "")
              nilai_kontrak = nilai_kontrak.replace('Rp. ', '').replace('Rp ','')
              if nilai_kontrak == "Nilai Kontrak belum dibuat":
                  nilai_kontrak2 = nilai_kontrak.replace('Nilai Kontrak belum dibuat', '')
                  nilai_kontrak_list.append(nilai_kontrak2)
              else:
                nilai_kontrak_list.append(nilai_kontrak)

              k_l_pd = area.select_one('td:nth-of-type(3)').text
              k_l_pd_list.append(k_l_pd)

              nama_tahapan = area.select_one('td:nth-of-type(4)').text
              nama_tahapan_list.append(nama_tahapan)

              get_data_halaman_pengumuman(kode_paket, detail_paket, tahun, tahun_null)

def get_data_halaman_pengumuman(kode_paket, detail_paket, tahun, tahun_null):
    link_pengumuman = "{}".format(link_lpse + "/"+ kode_paket +"/pengumumanlelang")
    try:
        driver.get(link_pengumuman)
        time.sleep(0.5)
        data_pengumuman = BeautifulSoup(driver.page_source, 'lxml')
        content_pengumuman = data_pengumuman.find('div', class_="content")

        if content_pengumuman == None:
          print("link error pengumuman = ", link_pengumuman)
          not_found()
          link_pengumuman_error_list.append(True)
          metode2 = detail_paket.split(' - ')[3]
          metode_pengadaan = metode2.split()[0:3]
          metode_pengadaan = ' '.join(metode_pengadaan)
          metode_pengadaan_list.append(metode_pengadaan)
          metode_evaluasi = metode2.split()[3:]
          metode_evaluasi = ' '.join(metode_evaluasi)
          metode_evaluasi_list.append(metode_evaluasi)
          if tahun_null:
            tahun_anggaran_list.append(tahun)

        else:
          link_pengumuman_error_list.append('')
          nilai_pengumuman(data_pengumuman, tahun_null)
    except:
        timeout_link_lpse.append(link_pengumuman)
      
      
      
def get_data_filterjumlah(jumlah):
    # Pilih jumlah data per halaman yg ingin ditampilkan
    paginasi = False
    driver.find_element(By.NAME,"tbllelang_length").send_keys(jumlah)
    if jumlah == 'Semua':
        jumlah_data = driver.find_element(By.XPATH,"//div[@class='dataTables_info']").text.split(' ')[-2].replace(',','')
        if int(jumlah_data) < 1000:
          time.sleep(10)
        elif int(jumlah_data) > 5000:
          time.sleep(40)
        else:
          time.sleep(20)
        
        data_tmp = BeautifulSoup(driver.page_source, 'lxml')
        tr_tender = data_tmp.select('table[id="tbllelang"] tbody tr')
        if len(tr_tender) == 25:
          paginasi = True
          jumlah = 100
    else:
        paginasi = True
        time.sleep(2)
    # Jumlah paginasi halaman
    page_list = driver.find_elements(By.XPATH, "//ul[@class='pagination']//li")
    last = int(page_list[-3].text)
    for i in tqdm (range(1,last+1)):
        get_data_halaman_tender()
        if paginasi:
            driver.get(link_lpse)
            time.sleep(2)
            driver.find_element(By.NAME,"tbllelang_length").send_keys(jumlah)
            time.sleep(2)
            for j in range(i):
                driver.execute_script("arguments[0].click()", driver.find_element(By.XPATH, "//li[@id='tbllelang_next']"))
                time.sleep(2)
        
        
def get_data_filtertahun(tahun, jumlah):
    # get website
    driver.get(link_lpse)
    # waiting to website load succesfully
    time.sleep(2)
    # Filter tahun sesuai parameter
    Select(driver.find_element(By.XPATH, "//select[@name='tahun']")).select_by_value(tahun)
    time.sleep(2)
    get_data_filterjumlah(jumlah)

In [25]:
err_link_lpse = []
timeout_link_lpse = []

In [34]:
#variabel data tender
link_lpse_list = []
kode_paket_list = []
nama_paket_list = []
status_list = []
jenis_pengadaan_list=[]
tahun_anggaran_list=[]
metode_pemilihan_list=[]
nilai_kontrak_list=[]
k_l_pd_list=[]
nama_tahapan_list=[]

#variabel data pengumuman
alasan_list=[]
kode_rup_list=[]
tanggal_pembuatan_list=[]
satuan_kerja_list=[]
metode_pengadaan_list=[]
metode_evaluasi_list=[]
sumber_dana_list=[]
nilai_pagu_list=[]
nilai_hps_list=[]
jenis_kontrak_list=[]
lokasi_pekerjaan_list=[]
kualifikasi_usaha_list=[]
syarat_kualifikasi_list=[]
jumlah_peserta_tender_list=[]

link_pengumuman_error_list=[]

In [35]:
# set Url based on index in dataset
i=119
end=120

# Looping to put data in variables
for i in tqdm(range(i, end)):
        try:
            # url from dataset add /lelang
            link_lpse = "{}".format(df_src['url'].iloc[i]+"/lelang")

            # get website
            driver.get(link_lpse)
            # waiting to website load succesfully
            time.sleep(2)
            tampilan = driver.find_elements(By.XPATH, "//select[@name='tbllelang_length']//option")
            jumlah_total = driver.find_element(By.XPATH,"//div[@class='dataTables_info']").text.split(' ')[-2].replace(',','')
            data_home_default = BeautifulSoup(driver.page_source, 'lxml')
            years = [e.text for e in data_home_default.select('select[name="tahun"] option')[1:-1]]
            years.sort(reverse=True)
            if years == [] or len(tampilan) < 4 :
                get_data_filterjumlah('100')
            else:
                for year in tqdm(years):
                  get_data_filtertahun(year, 'Semua')

            if len(link_lpse_list) != int(jumlah_total):
              driver.get(link_lpse)
              time.sleep(2)
              driver.find_element(By.NAME,"tbllelang_length").send_keys('Semua')
              if int(jumlah_total) < 5000:
                time.sleep(20)
              else:
                time.sleep(50)
              get_data_halaman_tender()

        except:
            # next index url
            err_link_lpse.append(link_lpse)
            pass
                    

  0%|          | 0/1 [00:00<?, ?it/s]

In [36]:
err_link_lpse

['https://lpse.jabarprov.go.id/eproc4/lelang',
 'http://lpse.gorutkab.go.id/eproc4/lelang',
 'http://lpse.gowakab.go.id/eproc4/lelang']

In [37]:
timeout_link_lpse

[]

In [17]:
tender_list =[]
spse_list =[]
info_list = []
for sublist in status_list:
    tmp_tender=[]
    tmp_spse=[]
    tmp_info=[]
    for e in sublist:
        if "Tender"in e:
            tmp_tender.append(e)
        elif "spse" in e:
            tmp_spse.append(e)
        else:
            tmp_info.append(e)
    tender_list.append(tmp_tender)
    spse_list.append(tmp_spse)
    info_list.append(tmp_info)

In [18]:
data_pengumuman_dict = {
    "url_lpse": link_lpse_list,
    "id_tender":kode_paket_list,
    "id_rup":kode_rup_list,
    "nama_tahapan": nama_tahapan_list,
    "status": tender_list,
    "status_spse": spse_list,
    "status_evaluasi":info_list,
    "alasan": alasan_list,
    "nama_paket": nama_paket_list,
    "sumber_dana": sumber_dana_list,
    "tgl_pembuatan":tanggal_pembuatan_list,
    "k_l_pd":k_l_pd_list,
    "satker": satuan_kerja_list,
    "jenis_tender": jenis_pengadaan_list,
    "metode_pengadaan": metode_pengadaan_list,
    "metode_pemilihan": metode_pemilihan_list,
    "metode_evaluasi": metode_evaluasi_list,
    "tahun_anggaran": tahun_anggaran_list,
    "nilai_kontrak": nilai_kontrak_list,
    "nilai_pagu": nilai_pagu_list,
    "nilai_hps": nilai_hps_list,
    "cara_bayar": jenis_kontrak_list,
    "lokasi_pekerjaan": lokasi_pekerjaan_list,
    "kualifikasi": kualifikasi_usaha_list,
    "detail_kualifikasi": syarat_kualifikasi_list,
    "jumlah_peserta": jumlah_peserta_tender_list,
    
    "link_pengumuman_error":link_pengumuman_error_list
}

In [19]:
import pandas as pd
df_pengumuman = pd.DataFrame.from_dict(data_pengumuman_dict, orient = 'index').T
pd.options.display.max_columns = 50
# pd.set_option('display.max_rows', None)

# Replace empty list with empty string
df_pengumuman = df_pengumuman.mask(df_pengumuman.applymap(str).eq('[]'))
df_pengumuman = df_pengumuman.fillna('')

# Hapus kurung siku
df_pengumuman = df_pengumuman.apply(lambda x: x.explode().astype(str).groupby(level=0).agg(", ".join))
df_pengumuman

,url_lpse,id_tender,id_rup,nama_tahapan,status,status_spse,status_evaluasi,alasan,nama_paket,sumber_dana,tgl_pembuatan,k_l_pd,satker,jenis_tender,metode_pengadaan,metode_pemilihan,metode_evaluasi,tahun_anggaran,nilai_kontrak,nilai_pagu,nilai_hps,cara_bayar,lokasi_pekerjaan,kualifikasi,detail_kualifikasi,jumlah_peserta,link_pengumuman_error
0,http://103.131.61.17/eproc4,3628631,36169184,Tender Sudah Selesai,,spse 4.5,,,Pengadaan Prasarana Air Bersih Rumah Sakit Ado...,APBD,26-09-2022,Pemerintah Daerah Kabupaten Flores Timur,Dinas Kesehatan,Pekerjaan Konstruksi,Pascakualifikasi Satu File,Tender,Harga Terendah Sistem Gugur,2022,,"550.000.000,00","549.500.000,00",Gabungan Lumsum dan Harga Satuan,Desa Saosina - Flores Timur (Kab.),Kecil,<strong>Persyaratan Kualifikasi Administrasi/L...,25 peserta,
1,http://103.131.61.17/eproc4,3624631,36415404,Tender Sudah Selesai,,spse 4.5,Evaluasi Ulang,,Pengadaan obat-obat lainnya non e-katalog,APBD,22-08-2022,Pemerintah Daerah Kabupaten Flores Timur,RSUD Larantuka Dr Hendrikus Fernandez,Pengadaan Barang,Pascakualifikasi Satu File,Tender,Harga Terendah Sistem Gugur,2022,,"395.141.681,00","395.141.681,00",Lumsum,Larantuka - Flores Timur (Kab.),Kecil,<strong>Persyaratan Kualifikasi Administrasi/L...,15 peserta,
2,http://103.131.61.17/eproc4,3622631,33010731,Tender Sudah Selesai,,spse 4.5,,,Pembangunan Pagar Pengaman Pekuburan Patigrama,APBD,12-08-2022,Pemerintah Daerah Kabupaten Flores Timur,Dinas Pekerjaan Umum dan Penataan Ruang,Pekerjaan Konstruksi,Pascakualifikasi Satu File,Tender,Harga Terendah Sistem Gugur,2022,,"270.000.000,00","269.998.000,00",Gabungan Lumsum dan Harga Satuan,Pekuburan Patigrama Kelurahan Amagarapati - Fl...,Kecil,<strong>Persyaratan Kualifikasi Administrasi/L...,24 peserta,
3,http://103.131.61.17/eproc4,3620631,35734902,Tender Sudah Selesai,,spse 4.5,,,Pengadaan Bibit Ternak Babi,APBD,12-07-2022,Pemerintah Daerah Kabupaten Flores Timur,Dinas Perkebunan dan Peternakan,Pengadaan Barang,Pascakualifikasi Satu File,Tender,Harga Terendah Sistem Gugur,2022,"309.067.500,00","319.375.000,00","319.014.500,00",Lumsum,"1. Kecamatan Larantuka (Kelurahan Sarotari, Sa...",Kecil,<strong>Persyaratan Kualifikasi Administrasi/L...,18 peserta,
4,http://103.131.61.17/eproc4,3619631,33444612,Tender Sudah Selesai,,spse 4.5,,,Penguatan Data Base dan Survey Kondisi Jalan,APBD,11-07-2022,Pemerintah Daerah Kabupaten Flores Timur,Dinas Pekerjaan Umum dan Penataan Ruang,Jasa Lainnya,Pascakualifikasi Satu File,Tender,Harga Terendah Sistem Gugur,2022,"299.774.925,00","300.000.000,00","299.996.920,00",Lumsum,Tersebar - Flores Timur (Kab.),Kecil,<strong>Persyaratan Kualifikasi Administrasi/L...,17 peserta,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1113,http://103.131.61.17/eproc4,6631,,Tender Gagal,Tender Gagal,spse 3,,"Tidak Ada yang Memasukan Dokumen Penawaran,",pembangunan rumah dinas camat adonara,APBD,27-06-2013,Pemerintah Daerah Kabupaten Flores Timur,Dinas Pekerjaan Umum Pertambangan dan Energi,Pekerjaan Konstruksi,Pascakualifikasi Satu File,Lelang Pemilihan Langsung,Harga Terendah Sistem Gugur,2013,,"300.000.000,00","297.000.000,00",Gabungan Lumsum dan Harga Satuan,lewotobi - Flores Timur (Kab.),Kecil,"<table class=""table table-sm""><tbody><tr valig...",8 peserta,
1114,http://103.131.61.17/eproc4,4631,,Tender Sudah Selesai,,spse 3,,,pembangunan rumah dinas camat lewolema,APBD,27-06-2013,Pemerintah Daerah Kabupaten Flores Timur,Dinas Pekerjaan Umum Pertambangan dan Energi,Pekerjaan Konstruksi,Pascakualifikasi Satu File,Lelang Pemilihan Langsung,Harga Terendah Sistem Gugur,2013,,"300.000.000,00","297.000.000,00",Gabungan Lumsum dan Harga Satuan,lewobele - Flores Timur (Kab.),Kecil,"<table class=""table table-sm""><tbody><tr valig...",9 peserta,
1115,http://103.131.61.17/eproc4,3631,,Tender Gagal,Tender Gagal,spse 3,,,Pengadaan Alat Berat Loader,,,Pemerintah Daerah Kabupaten Flores Timur,,Pengadaan Barang,,Lelang Sederhana,,2013,

In [20]:
df_pengumuman.loc[df_pengumuman['link_pengumuman_error'] == 'True'].reset_index(drop=True)

,url_lpse,id_tender,id_rup,nama_tahapan,status,status_spse,status_evaluasi,alasan,nama_paket,sumber_dana,tgl_pembuatan,k_l_pd,satker,jenis_tender,metode_pengadaan,metode_pemilihan,metode_evaluasi,tahun_anggaran,nilai_kontrak,nilai_pagu,nilai_hps,cara_bayar,lokasi_pekerjaan,kualifikasi,detail_kualifikasi,jumlah_peserta,link_pengumuman_error


In [21]:
with pd.ExcelWriter(f"D:/Data Engineer - Beecons/Document/pengumuman/pengumuman_{i}.xlsx", options={'strings_to_urls': False}) as writer:
        df_pengumuman.to_excel(writer, index=False)

C:\Users\Asus\AppData\Local\Temp/ipykernel_8048/2870151293.py:1: FutureWarning: Use of **kwargs is deprecated, use engine_kwargs instead.
  with pd.ExcelWriter(f"D:/Data Engineer - Beecons/Document/pengumuman/pengumuman_{i}.xlsx", options={'strings_to_urls': False}) as writer:


In [28]:
hay gays jadikali ini saya akan mencoba mengetik menggunakan keyboardmajhal dua ratus ribu milik daud yaa gays syyaaaa oke gys sampe disini saja yaaa mau solat dulu

SyntaxError: invalid syntax (Temp/ipykernel_8048/303996964.py, line 1)